# 使用预训练的特定任务模型（ TWITTER 评论）进行电影影评的情感分类

In [ ]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from datasets import load_dataset


In [ ]:
# 影评数据
data = load_dataset("rotten_tomatoes", cache_dir="./cache")

data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [ ]:
# 查看训练数据的第0， 1， 最后一个元素
data['train'][0, 1, -1]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 1, 0]}

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Labels: 0 -> Negative; 1 -> Neutral; 2 -> Positive
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(model_path, cache_dir="./model_cache")
# Text Classification	    AutoModelForSequenceClassification	情感分析、垃圾邮件识别、主题分类
# Causal Language Modeling	AutoModelForCausalLM	            文本续写、对话生成、代码生成（GPT/LLaMA 类）
# Masked Language Modeling	AutoModelForMaskedLM	            掩码填充、语义理解（BERT 类）
# Seq2Seq Language Modeling	AutoModelForSeq2SeqLM           	机器翻译、文本摘要、改写（T5/BART 类）
# Token Classification	    AutoModelForTokenClassification	    命名实体识别（NER）、词性标注
# Question Answering	    AutoModelForQuestionAnswering	    抽取式问答（如 SQuAD）
# Feature Extraction	    AutoModel	                        提取文本语义向量（无任务头）
model = AutoModelForSequenceClassification.from_pretrained(model_path, cache_dir="./model_cache")

pipe = pipeline(model = model_path, tokenizer = model_path, top_k = 2)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassifi

In [ ]:
# 结果是按照从大到小的顺序排列的
pipe(["I do not like this movie"])[0]

[{'label': 'negative', 'score': 0.9120093584060669},
 {'label': 'neutral', 'score': 0.07591026276350021}]

In [ ]:
# 结果是按照从大到小的顺序排列的
pipe(["I love this movie"])[0]

[{'label': 'positive', 'score': 0.9674931764602661},
 {'label': 'neutral', 'score': 0.027071893215179443}]

In [ ]:
from transformers.pipelines.pt_utils import KeyDataset
import numpy as np
# 进度条
from tqdm import tqdm

In [ ]:
y_pred = []
# KeyDataset: 从 dataset 中获取特定的 key
for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):
    label = output[0]["label"]
    if label == "neutral" :
        label = output[1]["label"]
    assignment = 0
    if label == "positive":
        assignment = 1
    y_pred.append(assignment)

 88%|████████▊ | 943/1066 [00:32<00:03, 32.22it/s]

In [ ]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    performance = classification_report(y_true, y_pred, target_names=["negative", "positive"])
    print(performance)

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

              precision    recall  f1-score   support

    negative       0.76      0.88      0.81       533
    positive       0.86      0.72      0.78       533

    accuracy                           0.80      1066
   macro avg       0.81      0.80      0.80      1066
weighted avg       0.81      0.80      0.80      1066

